In [1]:
import torch
from torch_geometric import datasets
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling
import matplotlib.pyplot as plt
# from tqdm import tqdm

from losses import kl_div_vmf, reconstruction_loss
from train_utils import get_edge_probs, plot_curves
from models import VGAE
from sklearn.metrics import roc_auc_score, average_precision_score

from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = datasets.Planetoid(root='/__data/Cora', name='Cora', split='public')

In [4]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [5]:
transform = RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False) # because we perform negative sampling at each epoch ourselves

In [6]:
train_data, val_data, test_data = transform(dataset[0])

In [7]:
print('train data', train_data)
print('val data', val_data)
print('test data', test_data)

print(dataset[0].edge_index.shape[1])
print(train_data.pos_edge_label.size(0)*2 + val_data.pos_edge_label.size(0) + val_data.neg_edge_label.size(0) + test_data.pos_edge_label.size(0) + test_data.neg_edge_label.size(0))

train data Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488])
val data Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[263], pos_edge_label_index=[2, 263], neg_edge_label=[263], neg_edge_label_index=[2, 263])
test data Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[527], neg_edge_label_index=[2, 527])
10556
10556


In [8]:
# train_data['neg_edge_label_index'] = negative_sampling(train_data.pos_edge_label_index)

In [9]:
feature_dim = dataset[0].x.shape[1]
print('input dim', feature_dim)

input dim 1433


## Check model forward pass

In [10]:
val_edge_label_index = torch.cat([val_data.pos_edge_label_index, val_data.neg_edge_label_index], dim=1)
val_edge_label_index.shape

torch.Size([2, 526])

In [11]:
n_vgae = VGAE(input_dim=feature_dim, latent_dim=16, dropout=0.0, latent_distr='normal').to(device)

out_n_vgae, mus_n_vgae, logsigmas2s_n_vgae  = n_vgae(val_data.x.to(device), val_edge_label_index.to(device))
print('out', out_n_vgae.shape)
print('mus', mus_n_vgae.shape)
print('logsigmas2s', logsigmas2s_n_vgae.shape)
labs, edge_probs = get_edge_probs(out_n_vgae, val_data.pos_edge_label_index, val_data.neg_edge_label_index)
print('edge probs', len(edge_probs)==len(labs))

out torch.Size([2708, 2708])
mus torch.Size([2708, 16])
logsigmas2s torch.Size([2708, 16])
edge probs True


In [11]:
d = 16
s_vgae = VGAE(input_dim=feature_dim, latent_dim=d, dropout=0.0, latent_distr='vMF').to(device)


out_s_vgae, mus_s_vgae, logkappas_s_vgae, ws, epss, bs = s_vgae(val_data.x.to(device), val_edge_label_index.to(device))
print('out', out_s_vgae.shape)
print('mus', mus_s_vgae.shape)
print('logkappas', logkappas_s_vgae.shape)
print('ws', ws.shape)
print('epss', epss.shape)
print('bs', bs.shape)
labs, edge_probs = get_edge_probs(out_s_vgae, val_data.pos_edge_label_index, val_data.neg_edge_label_index)
print('edge probs', len(edge_probs)==len(labs))

out torch.Size([2708, 2708])
mus torch.Size([2708, 16])
logkappas torch.Size([2708])
ws torch.Size([2708])
epss torch.Size([2708])
bs torch.Size([2708])
edge probs True


In [12]:
num_epochs = 200
lr = 0.01
optimizer = torch.optim.Adam(s_vgae.parameters(), lr=lr)

## Train $\mathcal{S}$-VGAE

In [13]:
metrics = { 'train_loss': [],
           'train_auc': [],
               'train_ap': [],
               'val_auc' : [],
               'val_ap' : []}
val_edge_label_index = torch.cat([val_data.pos_edge_label_index, val_data.neg_edge_label_index], dim=1)

for epoch in range(num_epochs):

    # negative sampling 
    train_neg_edge_index = negative_sampling(train_data.pos_edge_label_index)
    train_data['edge_label_index'] = torch.cat([train_data.pos_edge_label_index, train_neg_edge_index], dim=1)


    kl = kl_div_vmf.apply
    
    optimizer.zero_grad()
    
    output, mus, logkappas, ws, epss, bs = s_vgae(train_data.x.to(device), train_data.edge_label_index.to(device) )
    kappas = torch.exp(logkappas)

    # print('kappa', kappas)
    kl_loss = kl(kappas, mus) 
    loss_recon = reconstruction_loss(output, train_data.pos_edge_label_index, train_neg_edge_index, ws, kappas, bs, epss, mus.shape[1])
    loss = loss_recon + kl_loss
    # print('loss_recon', loss_recon)  
    # print('kl_loss', kl_loss)
    # print('loss', loss)
    
    loss.backward()

    optimizer.step()

    with torch.no_grad():
        metrics['train_loss'].append(loss.item())

        train_labs, edge_probs_train = get_edge_probs(output, train_data.pos_edge_label_index, train_neg_edge_index)
        metrics['train_auc'].append(roc_auc_score(train_labs, edge_probs_train))
        metrics['train_ap'].append(average_precision_score(train_labs, edge_probs_train))

        out_val, mus_val, logkappas_val, ws, epss, bs = s_vgae(val_data.x.to(device), val_edge_label_index.to(device))
        
        val_labs, edge_probs_val = get_edge_probs(out_val, val_data.pos_edge_label_index, val_data.neg_edge_label_index)
        metrics['val_auc'].append(roc_auc_score(val_labs, edge_probs_val))
        metrics['val_ap'].append(average_precision_score(val_labs, edge_probs_val))

        if epoch % 10 == 0:
            print('Epoch: {:3d}, VAL AUC: {:.4f}'.format(epoch, metrics['val_auc'][-1]))
            print(f'Epoch {epoch:3d}, TRAIN LOSS: {loss.item():.4f}')
            print('loss_recon', loss_recon.item())  
            print('kl_loss', kl_loss.item())


Epoch:   0, VAL AUC: 0.4778
Epoch   0, TRAIN LOSS: 1437.2271
loss_recon 7156.04541015625
kl_loss -5718.818359375
Epoch:  10, VAL AUC: 0.4881
Epoch  10, TRAIN LOSS: -62723.8359
loss_recon 6643.4443359375
kl_loss -69367.28125
Epoch:  20, VAL AUC: 0.4863
Epoch  20, TRAIN LOSS: -79092.3203
loss_recon 6500.638671875
kl_loss -85592.9609375
Epoch:  30, VAL AUC: 0.4825
Epoch  30, TRAIN LOSS: -83721.8828
loss_recon 6523.6181640625
kl_loss -90245.5
Epoch:  40, VAL AUC: 0.5240
Epoch  40, TRAIN LOSS: -87337.5938
loss_recon 6507.96533203125
kl_loss -93845.5625
Epoch:  50, VAL AUC: 0.4852
Epoch  50, TRAIN LOSS: -87318.9375
loss_recon 6507.2041015625
kl_loss -93826.140625
Epoch:  60, VAL AUC: 0.4707
Epoch  60, TRAIN LOSS: -88775.2422
loss_recon 6526.40625
kl_loss -95301.6484375
Epoch:  70, VAL AUC: 0.4924
Epoch  70, TRAIN LOSS: -88037.2188
loss_recon 6495.17578125
kl_loss -94532.390625
Epoch:  80, VAL AUC: 0.4983
Epoch  80, TRAIN LOSS: -88818.3750
loss_recon 6475.97265625
kl_loss -95294.34375
Epoch: 

In [14]:
import pickle
with open(f's_vgae_dim{d}_lr{lr}.pkl', 'wb') as f:
    pickle.dump(metrics, f)

FileNotFoundError: [Errno 2] No such file or directory: 'Slides\\figures\\learning_curves_s_vgae_d16_lr0.01.png'

In [15]:
figure_path = Path().parent / 'Slides' / 'figures'
plot_curves(metrics, str(figure_path / f'learning_curves_s_vgae_d{d}_lr{lr}.png'))

## Grid search 
as done in the article 

In [ ]:
from itertools import product
from pathlib import Path
import pickle

latent_dim = [8, 16, 32]
learning_rates = [.01, .005, .001]

results = Path('results')


In [ ]:
for d, lr in product(latent_dim, learning_rates):
    metrics =  {
        'train_loss' : [],
        'val_auc' : [],
        'val_ap' : []
    }

    val_edge_label_index = torch.cat([val_data.pos_edge_label_index, val_data.neg_edge_label_index], dim=1)

    for epoch in range(num_epochs):

        # negative sampling 
        train_neg_edge_index = negative_sampling(train_data.pos_edge_label_index)
        train_data['edge_label_index'] = torch.cat([train_data.pos_edge_label_index, train_neg_edge_index], dim=1)

        kl = kl_div_vmf.apply
        
        optimizer.zero_grad()
        
        output, mus, logkappas, ws, epss, bs = s_vgae(train_data.x.to(device), train_data.edge_label_index.to(device) )
        kappas = torch.exp(logkappas)
        kl_loss = kl(kappas, mus) 
        loss_recon = reconstruction_loss(output, train_data.pos_edge_label_index, train_neg_edge_index)
        loss = loss_recon + kl_loss
        # print('loss_recon', loss_recon)  
        # print('kl_loss', kl_loss)
        # print('loss', loss)
        
        loss.backward()

        optimizer.step()

        with torch.no_grad():
            metrics['train_loss'].append(loss.item())

            out_val, mus_val, logkappas_val, ws, epss, bs = s_vgae(val_data.x.to(device), val_edge_label_index.to(device))
            val_labs, edge_probs_val = get_edge_probs(out_val, val_data.pos_edge_label_index, val_data.neg_edge_label_index)
            metrics['val_auc'].append(roc_auc_score(val_labs, edge_probs_val))
            metrics['val_ap'].append(average_precision_score(val_labs, edge_probs_val))
            
            if epoch % 10 == 0:
                print('Epoch: {:3d}, VAL AUC: {:.4f}'.format(epoch, log_metrics['val_auc'][-1]))
                print(f'Epoch {epoch:3d}, TRAIN LOSS: {loss.item():.4f}')


with open(results / f's_vgae_dim{d}_lr{lr}.pkl', 'wb') as f:
    pickle.dump(metrics, f)
